In [1]:
ENV["JULIA_REVISE_POLL"] = "1"
using Revise

In [2]:
using Pkg
Pkg.activate("..")

  Activating environment at `~/.julia/dev/SymArrays/Project.toml`


In [3]:
using SymArrays
using BenchmarkTools
using TensorOperations
using Random

┌ Info: Precompiling SymArrays [c2b21f84-4a8e-4c96-99a2-2b79df568f75]
└ @ Base loading.jl:1342


In [5]:
N = 300
A = rand(N)
S = SymArray{(3,),Float64}(N,N,N);
S.data .= 1:length(S)
B = collect(S)
C1 = SymArray{(2,),Float64}(N,N)
C2 = copy(C1)
@tensor C3[j,k] := A[i]*B[i,j,k]
contract!(C1,A,S,Val(1),Val(1))
# this is the "hand-written" version where A has to be 1D
contract!(C2,A,S,Val(1))
@assert C1 ≈ C2
@assert C1 ≈ C3

In [6]:
@btime @tensor C3[j,k] = A[i]*B[i,j,k]
@btime contract!(C1,A,S,Val(1),Val(1))
@btime contract!(C1,A,S,Val(1));

  42.080 ms (1 allocation: 16 bytes)
  28.383 ms (6 allocations: 368 bytes)
  23.791 ms (0 allocations: 0 bytes)


In [7]:
N, M = 30, 40
A = rand(N)
S = SymArray{(2,1),Float64}(N,N,M)
S.data[:] .= 1:length(S)
B = collect(S)
C1 = SymArray{(1,1),Float64}(N,M)
C2 = collect(C1)
@tensor C3[j,k] := A[i]*B[i,j,k]
contract!(C1,A,S,Val(1),Val(1))
# this is the "hand-written" version where A has to be 1D
contract!(C2,A,S,Val(1))
@assert C1 ≈ C2
@assert C1 ≈ C3

In [8]:
@btime @tensor C3[j,k] = A[i]*B[i,j,k]
@btime contract!(C1,A,S,Val(1),Val(1))
@btime contract!(C2,A,S,Val(1));

  30.180 μs (1 allocation: 16 bytes)
  79.457 μs (6 allocations: 368 bytes)
  44.421 μs (0 allocations: 0 bytes)


In [9]:
N, M = 30, 40
A = rand(M)
S = SymArray{(2,1),Float64}(N,N,M);
S.data[:] .= 1:length(S)
B = collect(S)
C1 = SymArray{(2,),Float64}(N,N)
C2 = copy(C1)
@tensor C3[i,j] := A[k]*B[i,j,k]
contract!(C1,A,S,Val(1),Val(3))
# this is the "hand-written" version where A has to be 1D
contract!(C2,A,S,Val(3))
@assert C1 ≈ C2
@assert C1 ≈ C3

In [10]:
@btime @tensor C3[i,j] = B[i,j,k]*A[k]
@btime contract!(C1,A,S,Val(1),Val(3))
# this is the "hand-written" version where A has to be 1D
@btime contract!(C2,A,S,Val(3));

  29.027 μs (1 allocation: 16 bytes)
  23.411 μs (7 allocations: 368 bytes)
  3.229 μs (0 allocations: 0 bytes)


In [11]:
N1, N2, N3 = 10, 12, 13
A = rand(N1,N2,N3)
S = SymArray{(3,2,1),Float64}(N1,N1,N1,N2,N2,N3)
rand!(S.data)
# 
C11 = SymArray{(1,1,2,2,1),Float64}(N2,N3,N1,N1,N2,N2,N3)
C12 = copy(C11)
C13 = copy(C11)
contract!(C11,A,S,Val(1),Val(1))
contract!(C12,A,S,Val(1),Val(2))
contract!(C13,A,S,Val(1),Val(3))
@assert C11 == C12
@assert C11 == C13
C24 = SymArray{(1,1,3,1,1),Float64}(N1,N3,N1,N1,N1,N2,N3)
contract!(C24,A,S,Val(2),Val(4))
C25 = SymArray{(1,1,3,1,1),Float64}(N1,N3,N1,N1,N1,N2,N3)
contract!(C25,A,S,Val(2),Val(5))
@assert C24 == C25

C36 = SymArray{(1,1,3,2),Float64}(N1,N2,N1,N1,N1,N2,N2)
contract!(C36,A,S,Val(3),Val(6));

In [12]:
contract!(C24,A,S,Val(2),Val(4));
B = collect(S)
@tensor C24_AB[i,k,l,m,n,o,p] := A[i,j,k] * B[l,m,n,j,o,p]
@assert C24 ≈ C24_AB

In [13]:
contract!(C36,A,S,Val(3),Val(6));
B = collect(S)
@tensor C36_AB[i,j,l,m,n,o,p] := A[i,j,k] * B[l,m,n,o,p,k]
@assert C36 ≈ C36_AB

In [14]:
@tensor C11_AB[j,k,l,m,n,o,p] := A[i,j,k] * B[i,l,m,n,o,p]
@btime contract!(C11,A,S,Val(1),Val(1))
@assert C11 ≈ C11_AB
@btime @tensor C11_AB[j,k,l,m,n,o,p] = A[i,j,k] * B[i,l,m,n,o,p];

  71.254 ms (6 allocations: 368 bytes)
  65.991 ms (1 allocation: 16 bytes)


In [15]:
@btime contract!(C36,A,S,Val(3),Val(6));
@btime @tensor C36_AB[i,j,l,m,n,o,p] = A[i,j,k] * B[l,m,n,o,p,k];
@assert C36 ≈ C36_AB

  3.017 ms (7 allocations: 368 bytes)
  41.001 ms (1 allocation: 16 bytes)


In [16]:
@generated function benchtensor(res_B, A, B::Array{T,Ndim},::Val{mm}) where {T,Ndim,mm}
    inds_B = Symbol.(:i,1:Ndim)
    inds_B[mm] = :j
    inds_res = (:i,inds_B[1:Ndim .!= mm]...)
    :( @tensor res_B[$(inds_res...)] = A[i,j]*B[$(inds_B...)] )
end

NN = 4
maxNdim = 12
for Ndim = 2:2:maxNdim
    S = SymArray{(Ndim,),Float64}(ntuple(i->NN,Ndim)...)
    rand!(S.data)
    println(Ndim," ",length(S)," ",prod(size(S)))
    B = collect(S)

    A = rand(NN,NN)
    res = SymArray{(1,Ndim-1),Float64}(size(S)...)
    res_B = Array{Float64,Ndim}(undef,size(res))

    mm = Val(Ndim÷2)
    @btime contract!($res,$A,$S,Val(2),$mm)
    @btime benchtensor($res_B, $A, $B, $mm)
    
    @assert res ≈ res_B
end

2 10 16
  296.353 ns (6 allocations: 368 bytes)
  401.425 ns (1 allocation: 16 bytes)
4 35 256
  731.752 ns (6 allocations: 368 bytes)
  6.071 μs (34 allocations: 2.20 KiB)
6 84 4096
  2.825 μs (6 allocations: 368 bytes)
  18.796 μs (30 allocations: 2.33 KiB)
8 165 65536
  8.301 μs (6 allocations: 368 bytes)
  185.320 μs (32 allocations: 2.70 KiB)
10 286 1048576
  20.411 μs (6 allocations: 368 bytes)
  3.022 ms (105 allocations: 7.47 KiB)
12 455 16777216
  69.294 μs (6 allocations: 368 bytes)
  75.668 ms (180 allocations: 13.55 KiB)
